In [2]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.1)

#b = chat.predict("How mant planets are there?")


In [8]:
from langchain.schema import HumanMessage,AIMessage,SystemMessage
from langchain.prompts import PromptTemplate,ChatPromptTemplate

messages_1 = [
    #너는 일정 조율을 해주는 내 매니저 역할을 담당중
    SystemMessage(content="You are a grography expert. And you only relpy in Italian."),
    #안녕하세요 XXX입니다. 회의 햣 일정 회신 부탁드립니다.
    AIMessage(content="Ciao, mi chiamo Paolo"),
    #누구누구 연락처 그룹에게 4월 XX일부터 4월 XX일 까지 가능한 일정 회신 요청 문자를 보내줘
    HumanMessage(content="What is the distance between Mexico and Tailand? Also what is your name?")
]

#chat.predict_messages(messages_1)

#템플릿을 포맷해서 프롬프트로 만들어 담고 그 프롬프트롤 predict 메소드를 사용해 chat 모델에게 전달함
template_1 = PromptTemplate.from_template("What is the distance between {contry_a} and {contry_b}?")
prompt_1 = template_1.format(contry_a="Mexico",contry_b="Tailand")
#chat.predict(prompt)
#prompt_1

template = ChatPromptTemplate.from_messages([
    ("system","You are a geography expert. And You only reply in {language}."),
    ("ai","Ciao, mi chiamo {name}"),
    ("human","What is the distance between {country_a} and {country_b}. Also what is your name?"),
])

prompt = template.format_messages(
    language="Greek",
    name="Sora",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Άρης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [11]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
p = CommaOutputParser()
p.parse("Hello, how, are,you ?")


['Hello', 'how', 'are', 'you ?']

In [ ]:
from langchain.schema import HumanMessage,AIMessage,SystemMessage
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser

#chatmodel / OutputParser / Template 3가지로 간단히 만들기
chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))

template = ChatPromptTemplate.from_messages([
    ("system","You are a list generating machine. Everthing you are asked wiil be answered with a comma seperated list of max {max_items} in uppercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt3 = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

#result = chat.predict_messages(prompt3)
#p = CommaOutputParser()
#p.parse(result.content)

['RED',
 'BLUE',
 'GREEN',
 'YELLOW',
 'ORANGE',
 'PURPLE',
 'PINK',
 'BLACK',
 'WHITE',
 'BROWN']

In [ ]:
chain_one = template | chat | CommaOutputParser()
chain_one.invoke({
    "max_items":5, "question":"What are the pokemons?"
})

['PIKACHU', 'CHARIZARD', 'BULBASAUR', 'SQUIRTLE', 'EEVEE']